In [4]:
from lnas import LnasFormat
import pathlib
import numpy as np

NORMAL_OFFSET = 0.125

mesh_path = pathlib.Path("../insight/Docker/local/volume/PrologisCajamar4_full/G100/mesh/G100.lnas")
neg_mesh_path = mesh_path.parent / "G100_negative.lnas"
output_path = mesh_path.parent / "G100.merged.lnas"

mesh = LnasFormat.from_file(mesh_path)
neg_mesh = LnasFormat.from_file(neg_mesh_path)

geometry_negative = neg_mesh.geometry
geometry_negative.triangles = np.flip(geometry_negative.triangles, 1)
geometry_negative._full_update()
geometry_negative.vertices = geometry_negative.vertices + geometry_negative.vertices_normals * NORMAL_OFFSET

mesh.join(lnas_fmts=[neg_mesh], surfaces_suffixes=["_inside"])
mesh.to_file(output_path)

In [ ]:
import pandas as pd
import pathlib

data_path = pathlib.Path("../insight/Docker/local/volume/PrologisCajamar4_full/G100/cases/013/data/body_data")

body_hist_series = pd.read_hdf(data_path / "bodies.G100_hs.data.h5")
negative_body_hist_series = pd.read_hdf(data_path / "bodies.G100_neg_hs.data.h5")

last_index = body_hist_series.point_idx.max()
negative_body_hist_series.point_idx += last_index + 1

new_df = pd.concat([body_hist_series, negative_body_hist_series]).sort_values(
    by=["time_step", "point_idx"]
)
new_df.to_hdf(path_or_buf=data_path / "bodies.G100.merged.data.h5", key="df", mode="w")